# Olist : maintenance

In [1]:
#librairies import

import pandas as pd
import numpy as np
import scipy.stats as scp.sts
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans

from scp.sts import kstest, norm, expon, uniform

We will first use the ks test to evaluate if our data follows a certain logic

In [3]:
data = pd.read_csv('data/data_with_clustering.csv')
data.head()

,last_purchase,days_since_last_purchase,total_price_items,cluster,total_freight_value,total_orders,DBSCAN_cluter,HAC_cluter
0,1.534792e+09,2123,729.62,2,172.42,17,-1,0
1,1.519757e+09,2297,1000.85,2,171.82,9,-1,0
2,1.530147e+09,2176,664.20,2,94.63,7,-1,0
3,1.527853e+09,2203,806.61,2,316.11,7,-1,0
4,1.518615e+09,2310,855.21,2,165.91,7,-1,0


In [ ]:
#Model import

joblib_file = "kmeans_model.pkl"
kmeans_original = joblib.load(joblib_file)

need to import the last trained model as well

### ARI analysis

We will use the KS test to establish which statistical law is the fittest to simulate data evolution, then use the ARI to see when changes are to important, therefore calling for a retraining of the model

In [ ]:
'''# Perform K-S test for different distributions
ks_test_normal = kstest(data, 'norm', args=(data.mean(), data.std()))
ks_test_expon = kstest(data, 'expon', args=(data.min(), data.mean()))
ks_test_uniform = kstest(data, 'uniform', args=(data.min(), data.max() - data.min()))

print(f'K-S test for normal distribution: {ks_test_normal}')
print(f'K-S test for exponential distribution: {ks_test_expon}')
print(f'K-S test for uniform distribution: {ks_test_uniform}')'''

In [ ]:
'''#Depending on KS results

#norm
noise = np.random.normal(0, 0.5, data.shape)
evolved_data = data + noise

#expon
noise = np.random.exponential(0, 0.5, data.shape)
evolved_data = data + noise

#uniform
noise = np.random.uniform(0, 0.5, data.shape)
evolved_data = data + noise


# Perform clustering on both datasets
n_clusters = kmeans_original.n_clusters

kmeans_original = KMeans(n_clusters=n_clusters, random_state=42).fit(data)
kmeans_evolved = KMeans(n_clusters=n_clusters, random_state=42).fit(evolved_data)

# Calculate ARI
ari_score = adjusted_rand_score(kmeans_original.labels_, kmeans_evolved.labels_)
print(f'Adjusted Rand Index: {ari_score:.3f}')'''